In [ ]:
!pip install alt-profanity-check

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for alt-profanity-check: filename=alt_profanity_check-1.2.2-py3-none-any.whl size=1866162 sha256=0d3dd7447a9076cfa5607839665240a40376893395a4d3eb1950ca616bb3c68e
  Stored in directory: /root/.cache/pip/wheels/5b/98/77/c2903d8f2862ecf6ac3f51007e82f12d456f1ac7f6a147e7ab
Successfully built alt-profanity-check


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9e380e1623b7f35cfa4bcf672dba29ce5f86986f732f9b91cc80ec155fb2d795
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00


In [ ]:
import profanity_check
import re
# Define a sample text containing censored words
text = "You are a retard. I hate you. I hate the fact you exist. You are scum. You are an absolute fucking douchebag. You are a lowlife piece of human garbage. You absolute fucking retard cunt. You are a waste of a human being. You've done ABSOLUTELY NOTHING that's important. You dumb cunt, you piece of shit. You're not even a son of a bitch, that's too dignified a term for what you are. You know, you're not even human. You don't deserve the word human because you are that much of a fucking disgrace to people all around the globe, worldwide. It's a dishonor to put you in the same sentence with that word, you asshole. Are you fucking proud of yourself? Are you proud of having accomplished absolutely nothing in your life? You have only done bad things in this world. You do not deserve to live, you shit. You are a terrible excuse for a human being. You are an ABSOLUTE cunt. You are nothing but a faggot brainlet. You sleazy two-faced bastard. You slimy sluggish asshole. You TRIFLING motherfucker."
text1=re.sub(r"[^a-zA-Z0-9]"," ",text)
# Use the profanity_check library to identify censored words
censored_words = [word for word in text1.split() if profanity_check.predict([word])[0] == 1]
# Print the censored words
print(text1)
print("Censored words:", censored_words)


You are a retard  I hate you  I hate the fact you exist  You are scum  You are an absolute fucking douchebag  You are a lowlife piece of human garbage  You absolute fucking retard cunt  You are a waste of a human being  You ve done ABSOLUTELY NOTHING that s important  You dumb cunt  you piece of shit  You re not even a son of a bitch  that s too dignified a term for what you are  You know  you re not even human  You don t deserve the word human because you are that much of a fucking disgrace to people all around the globe  worldwide  It s a dishonor to put you in the same sentence with that word  you asshole  Are you fucking proud of yourself  Are you proud of having accomplished absolutely nothing in your life  You have only done bad things in this world  You do not deserve to live  you shit  You are a terrible excuse for a human being  You are an ABSOLUTE cunt  You are nothing but a faggot brainlet  You sleazy two faced bastard  You slimy sluggish asshole  You TRIFLING motherfucker 


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col

# Create a SparkSession object
spark = SparkSession.builder.appName("profanity check").getOrCreate()

# Convert the 1D list to a Spark DataFrame
df = spark.createDataFrame([(x,) for x in censored_words], schema=["text"])

# Split the text column into words and count the occurrences of each word
word_count_df= df.selectExpr("explode(split(text, ' ')) as word") \
                  .groupBy("word") \
                  .agg(count(col("word")).alias("count"))
# Create a temporary view of the DataFrame
word_count_df.createOrReplaceTempView("word_count")

# Query the table and show the result
result = spark.sql("SELECT * FROM word_count")
#result.unpersist()
result.show()
mostrepeated_word=spark.sql("select word from word_count where count=(select max(count) from word_count)")
mostrepeated_word.show()

+------------+-----+
|        word|count|
+------------+-----+
|   douchebag|    1|
|      retard|    2|
|        hate|    2|
|     garbage|    1|
|        dumb|    1|
|        scum|    1|
|     fucking|    4|
|        cunt|    3|
|motherfucker|    1|
|    disgrace|    1|
|       faced|    1|
|      faggot|    1|
|        shit|    2|
|     bastard|    1|
|     asshole|    2|
|       bitch|    1|
+------------+-----+

+-------+
|   word|
+-------+
|fucking|
+-------+



In [ ]:
from transformers import pipeline 
sentiment_pipeline=pipeline('sentiment-analysis')
results=sentiment_pipeline(text)
for result in results:
    print(f"Sentiment:{result['label']},Score:{result['score']}")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Sentiment:NEGATIVE,Score:0.999332845211029


In [ ]:
spark.sql('drop table word_count')
spark.sql('Show tables from default').show()